<a href="https://colab.research.google.com/github/DangMinh21/CS406-Image-Processing-and-Applications/blob/main/Deeplab/deeplab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Kết nối drive và import các thư viện cần thiết

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import tensorflow as tf
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, BatchNormalization, Input, DepthwiseConv2D, \
    add, Dropout, AveragePooling2D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.python.keras.layers import Layer, InputSpec
from tensorflow.python.keras.utils import conv_utils
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

# lấy data

In [9]:
SIZE = 256

In [10]:
def parse_data(X_path, y_path):
  arr = []
  for i in os.listdir(X_path):
    arr.append(i[:-4])
  list_X = []
  list_y = []
  for i in arr:
    list_X.append(X_path+'/'+i+'.jpg')
    list_y.append(y_path+'/'+i+'.png')
  return list_X, list_y

In [11]:
images_train_path, masks_train_path = parse_data('/content/drive/MyDrive/New/resized_data/train/photos','/content/drive/MyDrive/New/resized_data/train/masks')

In [12]:
images_val_path, masks_val_path = parse_data('/content/drive/MyDrive/New/resized_data/val/photos','/content/drive/MyDrive/New/resized_data/val/masks')

In [63]:
images_train = [cv2.imread(img, 0) for img in images_train_path]
masks_train = [cv2.imread(img, 0) for img in masks_train_path]
images_val = [cv2.imread(img, 0) for img in images_val_path]
masks_val = [cv2.imread(img, 0) for img in masks_val_path]

In [64]:
images_train = np.array(images_train)
images_train = np.expand_dims(images_train, axis=3)

masks_train = np.array(masks_train)
masks_train = np.expand_dims(masks_train, axis=3)

images_val = np.array(images_val)
images_val = np.expand_dims(images_val, axis=3)

masks_val = np.array(masks_val)
masks_val = np.expand_dims(masks_val, axis=3)

In [65]:
print("Image data shape is: ", images_val.shape)
print("Mask data shape is:", masks_val.shape)
print("Max pixel value in image is: ", images_val.max())
print("Labels in the mask are: ", np.unique(masks_val))

Image data shape is:  (210, 256, 256, 1)
Mask data shape is: (210, 256, 256, 1)
Max pixel value in image is:  255
Labels in the mask are:  [  0 255]


# Normalizations images

In [66]:
# Normalize images
images_train = images_train/255

masks_train = masks_train/255

images_val = images_val/255

masks_val = masks_val/255


In [67]:
IMAGE_HEIGHT = images_train.shape[1]
IMAGE_WIDTH = images_train.shape[2]
IMAGE_CHANNELS = images_train.shape[3]

input_shape = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)

In [68]:
IMAGE_SIZE = 256
NUM_CLASSES =2

#model

In [70]:
divisor = 2


class bilinear_upsampling(Layer):

    def __init__(self, up_sampling=(2, 2), data_format=None, **kwargs):
        super(bilinear_upsampling, self).__init__(**kwargs)
        self.data_format = conv_utils.normalize_data_format(data_format)
        self.up_sampling = conv_utils.normalize_tuple(up_sampling, 2, 'size')
        self.input_spec = InputSpec(ndim=4)

    def compute_output_shape(self, input_shape):
        height = self.up_sampling[0] * input_shape[1] if input_shape[1] is not None else None
        width = self.up_sampling[1] * input_shape[2] if input_shape[2] is not None else None
        return input_shape[0], height, width, input_shape[3]

    def call(self, inputs):
        return tf.image.resize(inputs, (int(inputs.shape[1] * self.up_sampling[0]),
                                                   int(inputs.shape[2] * self.up_sampling[1])))

    def get_config(self):
        config = {'size': self.up_sampling,
                  'data_format': self.data_format}
        base_config = super(bilinear_upsampling, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


def xception_downsample_block(x, channels, top_relu=False):
    # separable conv1
    if top_relu:
        x = Activation("relu")(x)
    x = DepthwiseConv2D((3, 3), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Conv2D(channels, (1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # separable conv2
    x = DepthwiseConv2D((3, 3), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Conv2D(channels, (1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # separable conv3
    x = DepthwiseConv2D((3, 3), strides=(2, 2), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Conv2D(channels, (1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    return x


def res_xception_downsample_block(x, channels):
    res = Conv2D(channels, (1, 1), strides=(2, 2), padding="same", use_bias=False)(x)
    res = BatchNormalization()(res)
    x = xception_downsample_block(x, channels)
    x = add([x, res])
    return x


def xception_block(x, channels):
    # separable conv1
    x = Activation("relu")(x)
    x = DepthwiseConv2D((3, 3), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Conv2D(channels, (1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)

    # separable conv2
    x = Activation("relu")(x)
    x = DepthwiseConv2D((3, 3), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Conv2D(channels, (1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)

    # separable conv3
    x = Activation("relu")(x)
    x = DepthwiseConv2D((3, 3), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Conv2D(channels, (1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    return x


def res_xception_block(x, channels):
    res = x
    x = xception_block(x, channels)
    x = add([x, res])
    return x


def aspp(x, input_shape, out_stride):
    b0 = Conv2D(256//divisor, (1, 1), padding="same", use_bias=False)(x)
    b0 = BatchNormalization()(b0)
    b0 = Activation("relu")(b0)

    b1 = DepthwiseConv2D((3, 3), dilation_rate=(6, 6), padding="same", use_bias=False)(x)
    b1 = BatchNormalization()(b1)
    b1 = Activation("relu")(b1)
    b1 = Conv2D(256//divisor, (1, 1), padding="same", use_bias=False)(b1)
    b1 = BatchNormalization()(b1)
    b1 = Activation("relu")(b1)

    b2 = DepthwiseConv2D((3, 3), dilation_rate=(12, 12), padding="same", use_bias=False)(x)
    b2 = BatchNormalization()(b2)
    b2 = Activation("relu")(b2)
    b2 = Conv2D(256//divisor, (1, 1), padding="same", use_bias=False)(b2)
    b2 = BatchNormalization()(b2)
    b2 = Activation("relu")(b2)

    b3 = DepthwiseConv2D((3, 3), dilation_rate=(18, 18), padding="same", use_bias=False)(x)
    b3 = BatchNormalization()(b3)
    b3 = Activation("relu")(b3)
    b3 = Conv2D(256//divisor, (1, 1), padding="same", use_bias=False)(b3)
    b3 = BatchNormalization()(b3)
    b3 = Activation("relu")(b3)

    out_shape = int(input_shape[0] / out_stride)
    b4 = AveragePooling2D(pool_size=(out_shape, out_shape))(x)
    b4 = Conv2D(256//divisor, (1, 1), padding="same", use_bias=False)(b4)
    b4 = BatchNormalization()(b4)
    b4 = Activation("relu")(b4)
    b4 = bilinear_upsampling((out_shape, out_shape))(b4)

    x = Concatenate()([b4, b0, b1, b2, b3])
    return x


def deeplabv3_plus(input_shape, out_stride=16, num_classes=19):
    img_input = Input(shape=input_shape)
    x = Conv2D(32//divisor, (3, 3), strides=(2, 2), padding="same", use_bias=False)(img_input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(64//divisor, (3, 3), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = res_xception_downsample_block(x, 128)

    res = Conv2D(256//divisor, (1, 1), strides=(2, 2), padding="same", use_bias=False)(x)
    res = BatchNormalization()(res)
    x = Activation("relu")(x)
    x = DepthwiseConv2D((3, 3), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Conv2D(256//divisor, (1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = DepthwiseConv2D((3, 3), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Conv2D(256//divisor, (1, 1), padding="same", use_bias=False)(x)
    skip = BatchNormalization()(x)
    x = Activation("relu")(skip)
    x = DepthwiseConv2D((3, 3), strides=(2, 2), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Conv2D(256//divisor, (1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = add([x, res])

    x = xception_downsample_block(x, 728//divisor, top_relu=True)

    for i in range(16):
        x = res_xception_block(x, 728//divisor)

    res = Conv2D(1024//divisor, (1, 1), padding="same", use_bias=False)(x)
    res = BatchNormalization()(res)
    x = Activation("relu")(x)
    x = DepthwiseConv2D((3, 3), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Conv2D(728//divisor, (1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = DepthwiseConv2D((3, 3), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Conv2D(1024//divisor, (1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = DepthwiseConv2D((3, 3), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Conv2D(1024//divisor, (1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = add([x, res])

    x = DepthwiseConv2D((3, 3), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Conv2D(1536//divisor, (1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = DepthwiseConv2D((3, 3), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Conv2D(1536//divisor, (1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = DepthwiseConv2D((3, 3), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Conv2D(2048//divisor, (1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # aspp
    x = aspp(x, input_shape, out_stride)
    x = Conv2D(256//divisor, (1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.9)(x)

    # decoder
    x = bilinear_upsampling((4, 4))(x)
    dec_skip = Conv2D(48//divisor, (1, 1), padding="same", use_bias=False)(skip)
    dec_skip = BatchNormalization()(dec_skip)
    dec_skip = Activation("relu")(dec_skip)
    x = Concatenate()([x, dec_skip])

    x = DepthwiseConv2D((3, 3), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(256//divisor, (1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = DepthwiseConv2D((3, 3), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(256//divisor, (1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_classes, (1, 1), padding="same")(x)
    x = bilinear_upsampling((4, 4))(x)
    model = Model(img_input, x)
    return model


model = deeplabv3_plus(input_shape,num_classes=2)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_20 (Conv2D)             (None, 128, 128, 16  144         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_18 (BatchN  (None, 128, 128, 16  64         ['conv2d_20[0][0]']              
 ormalization)                  )                                                           

#train

In [71]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=Adam(learning_rate=0.001),
                      loss=loss,
                      # loss=[categorical_focal_loss(alpha=.25, gamma=2)],
                      metrics=['accuracy'])

In [72]:
checkpoint_path = "/content/drive/MyDrive/New/checkpoint1/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                              save_weights_only=True,
                                              period=25,
                                              verbose=1)

history = model.fit(images_train, masks_train,batch_size=16, 
                    verbose=1,
                    epochs=20,
                    callbacks=[cp_callback],
                    validation_data=(images_val, masks_val),
                    shuffle=False)

Epoch 1/20
53/53 [==============================] - 50s 564ms/step - loss: 0.4270 - accuracy: 0.8056 - val_loss: 0.6402 - val_accuracy: 0.6628
Epoch 2/20
53/53 [==============================] - 25s 467ms/step - loss: 0.3281 - accuracy: 0.8552 - val_loss: 0.6511 - val_accuracy: 0.6628
Epoch 3/20
53/53 [==============================] - 25s 473ms/step - loss: 0.2814 - accuracy: 0.8780 - val_loss: 0.6859 - val_accuracy: 0.6628
Epoch 4/20
53/53 [==============================] - 26s 482ms/step - loss: 0.2519 - accuracy: 0.8925 - val_loss: 0.7082 - val_accuracy: 0.6628
Epoch 5/20
53/53 [==============================] - 25s 478ms/step - loss: 0.2269 - accuracy: 0.9045 - val_loss: 0.7593 - val_accuracy: 0.6628
Epoch 6/20
53/53 [==============================] - 25s 472ms/step - loss: 0.2121 - accuracy: 0.9111 - val_loss: 0.7179 - val_accuracy: 0.6628
Epoch 7/20
53/53 [==============================] - 25s 472ms/step - loss: 0.2096 - accuracy: 0.9115 - val_loss: 0.6014 - val_accuracy: 0.6628

#Lưu model

In [73]:
model.save('/content/drive/MyDrive/New/checkpoint1/model_deeplab_20epoch.hdf5')